# Pre-processing delle recensioni di TripAdvisor per Roma

In [1]:
import sys
sys.path.append('../src')
from tripadvisor_preprocessing import *
from preprocessing import *
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import string

In [2]:
df_it = keep_italian_reviews()

Numero di recensioni totali in italiano:  81901 
Numero di recensioni totali: 223089


### Definizione delle label di output

In [3]:
# Map rating
df_it['review_rating'] = df_it['review_rating'].apply(label)

Si osserva che il datates è sbilancianto in quanto l'84% delle recensioni è positiva. Al contrario, risultano equilibrate le recensioni neutre e negative.

In [4]:
df_it['review_rating'].value_counts()/len(df_it)*100

 1    84.439750
-1     8.097581
 0     7.462668
Name: review_rating, dtype: float64

### Normalizzazione del test della review

We remove the punctuation and lower case.

In [5]:
df_it['review_text'] = df_it['review_text'].apply(normalize)
df_it['review_title'] = df_it['review_title'].apply(normalize)

## Importiamo il vocabolario con la polarity delle parole 

Quindi, individuiamo le singole parole del lessico e procediamo con la seguente strategia.

__Le parole delle recensioni che fanno parte del lessico vengono sostituite dai rispettivi tag che indicano la polarità della parola.__

Si sceglie di operare questa sostituzione con l'obiettivo di semplificare al modello l'individuazione di parole positive e negative.

In [6]:
# We do the substitution both for the titles and the reviews
df_it['review_text_token'] = df_it['review_text'].apply(lambda x: x.split())
df_it['review_text_flag'] = df_it['review_text_token'].apply(substitute_label)

#### Ripeto le stesse operazioni sui titoli

In [7]:
df_it['review_title_token'] = df_it['review_title'].apply(lambda x: x.split())
df_it['review_title_flag'] = df_it['review_title_token'].apply(substitute_label)

#### Tokenize i titoli e le review

In [8]:
df_it['review_title_token_flag'] = df_it['review_title_flag'].apply(lambda x: x.split())
df_it['review_text_token_flag'] = df_it['review_text_flag'].apply(lambda x: x.split())

In [9]:
df_it.to_csv('../data/output/tripadvisor/preprocessed_tripadvisor_final.csv', sep = ';')

## Vettorizzazione dei testi
* Creazione del vocabolario che mappa i sincoli termini ai rispettivi indici

In [10]:
total_words_title = set([w for list_words in df_it['review_title_token_flag'] for w in list_words])
total_words_text = set([w for list_words in df_it['review_text_token_flag'] for w in list_words])

In [11]:
total_words = total_words_title.union(total_words_text)
print ('Number of unique words: {}'.format(len(total_words)))

Number of unique words: 60842


In [12]:
vocabolario_index = {w:idx for idx,w in enumerate(list(total_words))}
index_vocabolario = {idx:w for w, idx in vocabolario_index.items()}

* Salvataggio del vocabolario {parola:indice}

In [13]:
with open('../data/output/tripadvisor/vocabolario.json', 'w') as f:
    json.dump(vocabolario_index, f)

* Map le recensioni con il vocabolario.

In [14]:
df_it['review_text_token_flag_index'] = df_it['review_text_token_flag'].apply(replace_word_index_tripadvisor)

* Salva dataset con sequenze

__!!!!Tenere solo le review positive o negative!!!!__

In [15]:
to_save = df_it[(df_it['review_rating'] == 1) | (df_it['review_rating'] == -1)][['review_rating','review_text_token_flag_index']]

In [16]:
to_save.head()

,review_rating,review_text_token_flag_index
0,1,"[18128, 51904, 8173, 45643, 18968, 8173, 8173,..."
3,1,"[8173, 21264, 18968, 8173, 46412, 35261, 8173,..."
5,1,"[18128, 51904, 8173, 45643, 18968, 8173, 8173,..."
8,1,"[8173, 21264, 18968, 8173, 46412, 35261, 8173,..."
10,1,"[2030, 16209, 11868, 35261, 55339, 26963, 5001..."


In [17]:
to_save.to_csv('../data/output/tripadvisor/dataset_sequence.csv', sep = ';')